In [82]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, optimizers, regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import time
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, Callback
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l1_l2
import keras
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [69]:
import zipfile
with zipfile.ZipFile('valoracion_aerolineas.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

df_train = pd.read_csv('/content/dataset/train.csv')
df_test = pd.read_csv('/content/dataset/test.csv')

In [70]:
df_train = df_train.dropna()
df_train = df_train.drop(columns=['Unnamed: 0', 'id'])
df_test = df_test.dropna()
df_test = df_test.drop(columns=['Unnamed: 0', 'id'])
df_train

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103899,Female,disloyal Customer,23,Business travel,Eco,192,2,1,2,3,...,2,3,1,4,2,3,2,3,0.0,neutral or dissatisfied
103900,Male,Loyal Customer,49,Business travel,Business,2347,4,4,4,4,...,5,5,5,5,5,5,4,0,0.0,satisfied
103901,Male,disloyal Customer,30,Business travel,Business,1995,1,1,1,3,...,4,3,2,4,5,5,4,7,14.0,neutral or dissatisfied
103902,Female,disloyal Customer,22,Business travel,Eco,1000,1,1,1,5,...,1,4,5,1,5,4,1,0,0.0,neutral or dissatisfied


In [71]:
df_train_dummie = df_train.copy()

# Aplicar Label Encoding a las columnas categóricas
label_encoder = LabelEncoder()
for column in df_train_dummie.select_dtypes(include=['object']).columns:
    df_train_dummie[column] = label_encoder.fit_transform(df_train_dummie[column])

df_test_dummie = df_test.copy()

# Aplicar Label Encoding a las columnas categóricas
label_encoder = LabelEncoder()
for column in df_test_dummie.select_dtypes(include=['object']).columns:
    df_test_dummie[column] = label_encoder.fit_transform(df_test_dummie[column])

In [72]:
df_train_selected = df_train_dummie[['Type of Travel','Class','Online boarding','satisfaction']]
df_test_selected = df_test_dummie[['Type of Travel','Class','Online boarding','satisfaction']]

In [74]:
# Age cluster
df_train_new_features = pd.DataFrame()
age_bins = [7,18,26,33,40,47,54,63,86]
df_train_new_features['Age Cluster'] = pd.cut(df_train_dummie['Age'], bins=age_bins, labels=False, right=False)

# Weight Comfort Seats
df_train_new_features['Weight Comfort Seats'] = (df_train_dummie['Seat comfort']/5 + df_train_dummie['Class'] + df_train_dummie['Type of Travel'])

# Media de todos los servicios que tienen valor del 0-5
df_train_new_features['Mean Satisfaction Services'] = df_train_dummie[['Inflight wifi service','Departure/Arrival time convenient','Ease of Online booking','Gate location','Food and drink',
                                  'Online boarding','Seat comfort','Inflight entertainment','On-board service','Leg room service','Baggage handling','Checkin service','Inflight service','Cleanliness']].mean(axis=1)

# Suma del Servicio en Vuelo
df_train_new_features['Sum Inflight Services'] = df_train_dummie['Inflight wifi service'] + df_train_dummie['Inflight service'] + df_train_dummie['Inflight entertainment'] + df_train_dummie['Online boarding']

# Peso sobre el espacio de los pies según la clase
df_train_new_features['Space Seat and Class'] = (df_train_dummie['Class'] * df_train_dummie['Leg room service']) / 5

# suma de servicios básicos
df_train_new_features['Weight Basic Services'] = df_train_dummie['Class'] + (df_train_dummie['Food and drink'] + df_train_dummie['Cleanliness'])/10


In [75]:
# Age clister
df_test_new_features = pd.DataFrame()
age_bins = [7,18,26,33,40,47,54,63,86]
df_test_new_features['Age Cluster'] = pd.cut(df_test['Age'], bins=age_bins, labels=False, right=False)

# Weight Comfort Seats
df_test_new_features['Weight Comfort Seats'] = (df_test_dummie['Seat comfort']/5 + df_test_dummie['Class'] + df_test_dummie['Type of Travel'])

# Media de todos los servicios que tienen valor del 0-5
df_test_new_features['Mean Satisfaction Services'] = df_test_dummie[['Inflight wifi service','Departure/Arrival time convenient','Ease of Online booking','Gate location','Food and drink',
                                  'Online boarding','Seat comfort','Inflight entertainment','On-board service','Leg room service','Baggage handling','Checkin service','Inflight service','Cleanliness']].mean(axis=1)

# Suma del Servicio en Vuelo
df_test_new_features['Sum Inflight Services'] = df_test_dummie['Inflight wifi service'] + df_test_dummie['Inflight service'] + df_test_dummie['Inflight entertainment'] + df_test_dummie['Online boarding']

# Peso sobre el espacio de los pies según la clase
df_test_new_features['Space Seat and Class'] = (df_test_dummie['Class'] * df_test_dummie['Leg room service']) / 5

# suma de servicios básicos
df_test_new_features['Weight Basic Services'] = df_test_dummie['Class'] + (df_test_dummie['Food and drink'] + df_test_dummie['Cleanliness'])/10

In [76]:
df_train_combined = pd.concat([df_train_selected, df_train_new_features], axis=1)

# Eliminar las columnas duplicadas si es necesario
df_train_combined = df_train_combined.loc[:, ~df_train_combined.columns.duplicated()]

df_test_combined = pd.concat([df_test_selected, df_test_new_features], axis=1)
df_test_combined = df_test_combined.loc[:, ~df_test_combined.columns.duplicated()]

# Callbacks

In [77]:
# Definir un callback personalizado para medir el tiempo de entrenamiento
class TimeHistory(Callback):
    def on_train_begin(self, logs=None):
        self.times = []
        self.train_start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        self.times.append(time.time() - self.epoch_start_time)

    def on_train_end(self, logs=None):
        self.total_train_time = time.time() - self.train_start_time
        
time_callback = TimeHistory()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)

# Arquitectura 1: Valores sin análisis previo
- Inputs: Todo el data set (dummie)
- Epochs = 100
- 1ra capa = 64, relu
- 2da capa = 32 relu
- 3ra capa = 1 sigmoid
- learning_rate=0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32



In [96]:
X_train = df_train_dummie.drop('satisfaction', axis=1)
y_train = df_train_dummie['satisfaction']

X_test = df_test_dummie.drop('satisfaction', axis=1)
y_test = df_test_dummie['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Capas
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Hiperparámetros
model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

#Recalls
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m1.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

# Modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 3s 978us/step - loss: 0.1994 - accuracy: 0.9190 - val_loss: 0.1414 - val_accuracy: 0.9401 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 2s 895us/step - loss: 0.1358 - accuracy: 0.9442 - val_loss: 0.1196 - val_accuracy: 0.9499 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 4s 2ms/step - loss: 0.1173 - accuracy: 0.9513 - val_loss: 0.1120 - val_accuracy: 0.9547 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 9s 3ms/step - loss: 0.1076 - accuracy: 0.9548 - val_loss: 0.1060 - val_accuracy: 0.9543 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 9s 4ms/step - loss: 0.1022 - accuracy: 0.9568 - val_loss: 0.1014 - val_accuracy: 0.9590 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 5s 2ms/step - loss: 0.0971 - accuracy: 0.9588 - val_loss: 0.0975 - val_accuracy: 0.9586 - lr: 0.0010
Epoch 7/50
2590/2590 [==============================] 

In [98]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m1.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 129.60 segundos
Pérdida: 0.0660059005022049
Accuracy Modelo: 97.14509844779968
Recall: 0.9456225252969643
Specificity: 0.5707334
F1-Score: 0.9561387900355872
Precisión: 0.9668915879442195


# Arquitectura 2: Correlativa Individual
- Inputs: 'Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service',"Cleanliness"
- Epochs = 50
- 1ra capa = 64, relu
- 2da capa = 32 relu
- 4ta capa = 1 sigmoid
- learning_rate=0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32

In [106]:
X_train = df_train_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_train = df_train_dummie['satisfaction']

X_test = df_test_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_test = df_test_dummie['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m2.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 3s 944us/step - loss: 0.2931 - accuracy: 0.8818 - val_loss: 0.2595 - val_accuracy: 0.8973 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 2s 888us/step - loss: 0.2601 - accuracy: 0.8950 - val_loss: 0.2463 - val_accuracy: 0.9022 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 3s 1ms/step - loss: 0.2514 - accuracy: 0.8979 - val_loss: 0.2415 - val_accuracy: 0.9021 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 2s 899us/step - loss: 0.2456 - accuracy: 0.9007 - val_loss: 0.2400 - val_accuracy: 0.9025 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 2s 915us/step - loss: 0.2404 - accuracy: 0.9016 - val_loss: 0.2357 - val_accuracy: 0.9044 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 2s 904us/step - loss: 0.2372 - accuracy: 0.9026 - val_loss: 0.2315 - val_accuracy: 0.9049 - lr: 0.0010
Epoch 7/50
2590/2590 [==========================

In [107]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m2.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 116.46 segundos
Pérdida: 0.2061804085969925
Accuracy Modelo: 91.41477942466736
Recall: 0.8807743070831501
Specificity: 0.57548374
F1-Score: 0.8954689806324642
Precisión: 0.9106622998544396


# Arquitectura 3: Correlaciones cercanas > 0.3 (+1 capa)
- Inputs: 'Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service',"Cleanliness"
- Epochs = 50
- 1ra capa = 64, relu
- 2da capa = 32 relu
- 3ra capa = 16 relu
- 4ta capa = 1 sigmoid
- learning_rate = 0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32

In [108]:
X_train = df_train_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_train = df_train_dummie['satisfaction']

X_test = df_test_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_test = df_test_dummie['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m3.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 3s 981us/step - loss: 0.2894 - accuracy: 0.8826 - val_loss: 0.2538 - val_accuracy: 0.8986 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 2s 911us/step - loss: 0.2577 - accuracy: 0.8966 - val_loss: 0.2425 - val_accuracy: 0.9027 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 2s 932us/step - loss: 0.2478 - accuracy: 0.8998 - val_loss: 0.2486 - val_accuracy: 0.8993 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 2s 946us/step - loss: 0.2407 - accuracy: 0.9020 - val_loss: 0.2305 - val_accuracy: 0.9071 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 3s 1ms/step - loss: 0.2354 - accuracy: 0.9039 - val_loss: 0.2290 - val_accuracy: 0.9065 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 3s 994us/step - loss: 0.2323 - accuracy: 0.9049 - val_loss: 0.2243 - val_accuracy: 0.9091 - lr: 0.0010
Epoch 7/50
2590/2590 [==========================

In [109]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m3.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 103.97 segundos
Pérdida: 0.20288875699043274
Accuracy Modelo: 91.50407314300537
Recall: 0.8815662120545534
Specificity: 0.5758699
F1-Score: 0.8966751689264779
Precisión: 0.9123110544527409


# Arquitectura 4: Correlaciones > 0.3 (Other activations)
- Inputs: 'Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service',"Cleanliness"
- Epochs = 50
- 1ra capa = 64, relu
- 2da capa = 32 tanh
- 4ta capa = 1 sigmoid
- learning_rate = 0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32

In [110]:
X_train = df_train_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_train = df_train_dummie['satisfaction']

X_test = df_test_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_test = df_test_dummie['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='tanh'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m4.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 3s 885us/step - loss: 0.2997 - accuracy: 0.8778 - val_loss: 0.2665 - val_accuracy: 0.8943 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 2s 890us/step - loss: 0.2654 - accuracy: 0.8923 - val_loss: 0.2490 - val_accuracy: 0.9006 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 3s 1ms/step - loss: 0.2541 - accuracy: 0.8969 - val_loss: 0.2430 - val_accuracy: 0.9012 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 3s 1ms/step - loss: 0.2462 - accuracy: 0.9006 - val_loss: 0.2363 - val_accuracy: 0.9046 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 2s 883us/step - loss: 0.2411 - accuracy: 0.9016 - val_loss: 0.2343 - val_accuracy: 0.9038 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 2s 860us/step - loss: 0.2376 - accuracy: 0.9028 - val_loss: 0.2301 - val_accuracy: 0.9049 - lr: 0.0010
Epoch 7/50
2590/2590 [============================

In [111]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m4.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 165.34 segundos
Pérdida: 0.2119370996952057
Accuracy Modelo: 91.231369972229
Recall: 0.8745270567531896
Specificity: 0.5805816
F1-Score: 0.894398200224972
Precisión: 0.9151933701657459


# Arquitectura 5: Correlaciones > 0.3 (Más nodos, más capas, nuevas funciones activación)
- Inputs: 'Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service',"Cleanliness"
- Epochs = 50
- 1ra capa = 128, relu
- 2da capa = 64 relu
- 3ra capa = 32 tanh
- 4ta capa = 1 sigmoid
- learning_rate = 0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32

In [112]:
X_train = df_train_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_train = df_train_dummie['satisfaction']

X_test = df_test_dummie[['Type of Travel', 'Class','Online boarding', 'Seat comfort', 'Inflight entertainment','On-board service','Leg room service','Cleanliness']]
y_test = df_test_dummie['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='tanh'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m5.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 3s 945us/step - loss: 0.2755 - accuracy: 0.8882 - val_loss: 0.2439 - val_accuracy: 0.9001 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 2s 909us/step - loss: 0.2461 - accuracy: 0.8996 - val_loss: 0.2320 - val_accuracy: 0.9072 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 3s 1ms/step - loss: 0.2365 - accuracy: 0.9021 - val_loss: 0.2259 - val_accuracy: 0.9047 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 2s 953us/step - loss: 0.2311 - accuracy: 0.9042 - val_loss: 0.2223 - val_accuracy: 0.9100 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 2s 952us/step - loss: 0.2266 - accuracy: 0.9060 - val_loss: 0.2205 - val_accuracy: 0.9107 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 3s 974us/step - loss: 0.2241 - accuracy: 0.9074 - val_loss: 0.2194 - val_accuracy: 0.9102 - lr: 0.0010
Epoch 7/50
2590/2590 [==========================

In [113]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m5.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 146.32 segundos
Pérdida: 0.19842857122421265
Accuracy Modelo: 91.65852069854736
Recall: 0.8780466344038715
Specificity: 0.5789982
F1-Score: 0.8963442019222132
Precisión: 0.9154206036143473


# Arquitectura 6: Nuevos features
- Epochs = 50
- 1raa capa = 64 relu
- 2da capa = 32 relu
- 3ra capa = 1 sigmoid
- learning_rate = 0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32

In [114]:
X_train = df_train_combined[['Age Cluster', 'Weight Comfort Seats', 'Mean Satisfaction Services',
       'Sum Inflight Services', 'Space Seat and Class',
       'Weight Basic Services']]
y_train = df_train_combined['satisfaction']

X_test = df_test_combined[['Age Cluster', 'Weight Comfort Seats', 'Mean Satisfaction Services',
       'Sum Inflight Services', 'Space Seat and Class',
       'Weight Basic Services']]
y_test = df_test_combined['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m6.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 3s 1ms/step - loss: 0.3700 - accuracy: 0.8458 - val_loss: 0.3453 - val_accuracy: 0.8609 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 2s 952us/step - loss: 0.3387 - accuracy: 0.8585 - val_loss: 0.3236 - val_accuracy: 0.8642 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 2s 912us/step - loss: 0.3259 - accuracy: 0.8637 - val_loss: 0.3109 - val_accuracy: 0.8698 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 2s 881us/step - loss: 0.3183 - accuracy: 0.8672 - val_loss: 0.3056 - val_accuracy: 0.8736 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 2s 866us/step - loss: 0.3129 - accuracy: 0.8703 - val_loss: 0.3051 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 2s 922us/step - loss: 0.3090 - accuracy: 0.8714 - val_loss: 0.2966 - val_accuracy: 0.8799 - lr: 0.0010
Epoch 7/50
2590/2590 [==========================

In [115]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m6.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 238.06 segundos
Pérdida: 0.27842628955841064
Accuracy Modelo: 88.37043642997742
Recall: 0.8303563572371315
Specificity: 0.5869153
F1-Score: 0.8555369203571914
Precisión: 0.8822924457741211


# Arquitectura 7: Nuevos Features (+1 capa)
- Epochs = 50
- 1ra capa = 64 relu
- 2da capa = 64 relu
- 3ra capa = 32 relu
- 4ta capa = 1 sigmoid
- learning_rate = 0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32

In [116]:
X_train = df_train_combined[['Age Cluster', 'Weight Comfort Seats', 'Mean Satisfaction Services',
       'Sum Inflight Services', 'Space Seat and Class',
       'Weight Basic Services']]
y_train = df_train_combined['satisfaction']

X_test = df_test_combined[['Age Cluster', 'Weight Comfort Seats', 'Mean Satisfaction Services',
       'Sum Inflight Services', 'Space Seat and Class',
       'Weight Basic Services']]
y_test = df_test_combined['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m7.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3537 - accuracy: 0.8528 - val_loss: 0.3221 - val_accuracy: 0.8665 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3197 - accuracy: 0.8675 - val_loss: 0.3166 - val_accuracy: 0.8696 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3095 - accuracy: 0.8718 - val_loss: 0.3004 - val_accuracy: 0.8768 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3044 - accuracy: 0.8734 - val_loss: 0.2934 - val_accuracy: 0.8783 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 5s 2ms/step - loss: 0.3021 - accuracy: 0.8737 - val_loss: 0.2952 - val_accuracy: 0.8770 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.2992 - accuracy: 0.8754 - val_loss: 0.2888 - val_accuracy: 0.8829 - lr: 0.0010
Epoch 7/50
2590/2590 [==============================] - 6s

In [117]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m7.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 341.09 segundos
Pérdida: 0.2589331567287445
Accuracy Modelo: 89.18612599372864
Recall: 0.834931808183018
Specificity: 0.5864133
F1-Score: 0.8597444957868986
Precisión: 0.8860771313848165


# Arquitectura 8: Nuevos Features (+1 capa) (dif activación)
- Epochs = 50
- 1ra capa = 128 relu
- 2da capa = 64 tanh
- 3ra capa = 32 tanh
- 4ta capa = 1 sigmoid
- learning_rate = 0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32

In [127]:
X_train = df_train_combined[['Age Cluster', 'Weight Comfort Seats', 'Mean Satisfaction Services',
       'Sum Inflight Services', 'Space Seat and Class',
       'Weight Basic Services']]
y_train = df_train_combined['satisfaction']

X_test = df_test_combined[['Age Cluster', 'Weight Comfort Seats', 'Mean Satisfaction Services',
       'Sum Inflight Services', 'Space Seat and Class',
       'Weight Basic Services']]
y_test = df_test_combined['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='tanh'),
    layers.Dense(32, activation='tanh'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m8.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3582 - accuracy: 0.8493 - val_loss: 0.3231 - val_accuracy: 0.8681 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3258 - accuracy: 0.8642 - val_loss: 0.3110 - val_accuracy: 0.8710 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3147 - accuracy: 0.8706 - val_loss: 0.3105 - val_accuracy: 0.8720 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3086 - accuracy: 0.8720 - val_loss: 0.3007 - val_accuracy: 0.8755 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3051 - accuracy: 0.8737 - val_loss: 0.2922 - val_accuracy: 0.8811 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3025 - accuracy: 0.8742 - val_loss: 0.2956 - val_accuracy: 0.8772 - lr: 0.0010
Epoch 7/50
2590/2590 [==============================] - 6s

In [128]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m8.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 334.95 segundos
Pérdida: 0.2604016363620758
Accuracy Modelo: 89.16682004928589
Recall: 0.842146942366916
Specificity: 0.5801954
F1-Score: 0.8608949853834045
Precisión: 0.8804967801287948


# Arquitectura 9: Nuevos features + Variables individuales
- Epochs = 50
- 1ra capa = 64 relu
- 2da capa = 32 relu
- 3ra capa = 1 sigmoid
- learning_rate = 0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32

In [131]:
X_train = df_train_combined.drop(columns='satisfaction')
y_train = df_train_combined['satisfaction']

X_test = df_test_combined.drop(columns='satisfaction')
y_test = df_test_combined['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m9.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.3059 - accuracy: 0.8747 - val_loss: 0.2730 - val_accuracy: 0.8909 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.2684 - accuracy: 0.8908 - val_loss: 0.2599 - val_accuracy: 0.8914 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 5s 2ms/step - loss: 0.2565 - accuracy: 0.8960 - val_loss: 0.2498 - val_accuracy: 0.8983 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 5s 2ms/step - loss: 0.2489 - accuracy: 0.9000 - val_loss: 0.2376 - val_accuracy: 0.9066 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.2432 - accuracy: 0.9021 - val_loss: 0.2372 - val_accuracy: 0.9037 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 5s 2ms/step - loss: 0.2387 - accuracy: 0.9043 - val_loss: 0.2346 - val_accuracy: 0.9048 - lr: 0.0010
Epoch 7/50
2590/2590 [==============================] - 5s

In [132]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m9.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 294.37 segundos
Pérdida: 0.20435096323490143
Accuracy Modelo: 91.82745218276978
Recall: 0.873295204575451
Specificity: 0.5826671
F1-Score: 0.8953136980740607
Precisión: 0.9184712196927632


# Arquitectura 10: Nuevos features + Variables individuales (Aumentando epochs)
- Epochs = 100
- 1ra capa = 64 relu
- 2da capa = 32 relu
- 3ra capa = 1 sigmoid
- learning_rate = 0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32

In [135]:
X_train = df_train_combined.drop(columns='satisfaction')
y_train = df_train_combined['satisfaction']

X_test = df_test_combined.drop(columns='satisfaction')
y_test = df_test_combined['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='tanh'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m10.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 7s 2ms/step - loss: 0.3021 - accuracy: 0.8749 - val_loss: 0.2706 - val_accuracy: 0.8911 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.2701 - accuracy: 0.8900 - val_loss: 0.2606 - val_accuracy: 0.8943 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 8s 3ms/step - loss: 0.2598 - accuracy: 0.8948 - val_loss: 0.2624 - val_accuracy: 0.8943 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 8s 3ms/step - loss: 0.2542 - accuracy: 0.8982 - val_loss: 0.2536 - val_accuracy: 0.8964 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 8s 3ms/step - loss: 0.2484 - accuracy: 0.9006 - val_loss: 0.2495 - val_accuracy: 0.9000 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 5s 2ms/step - loss: 0.2439 - accuracy: 0.9019 - val_loss: 0.2331 - val_accuracy: 0.9058 - lr: 0.0010
Epoch 7/50
2590/2590 [==============================] - 5s

In [136]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m10.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 355.78 segundos
Pérdida: 0.2001865804195404
Accuracy Modelo: 91.92277789115906
Recall: 0.8763748350197976
Specificity: 0.5817789
F1-Score: 0.8975398756420654
Precisión: 0.9197525163911718


# Arquitectura 11: Nuevos features + Variables individuales (Aumentando epochs)
- Epochs = 100
- 1ra capa = 64 relu
- 2da capa = 32 relu
- 3ra capa = 1 sigmoid
- learning_rate = 0.001
- optimazador = Adam
- loss = 'binary_crossentropy'
- batch_size=32

In [137]:
X_train = df_train_combined.drop(columns='satisfaction')
y_train = df_train_combined['satisfaction']

X_test = df_test_combined.drop(columns='satisfaction')
y_test = df_test_combined['satisfaction']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='tanh'),
    layers.Dense(32, activation='tanh'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('../MODELS/m11.h5', save_best_only=True, monitor='val_loss')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[time_callback, early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.2956 - accuracy: 0.8769 - val_loss: 0.2649 - val_accuracy: 0.8920 - lr: 0.0010
Epoch 2/50
2590/2590 [==============================] - 5s 2ms/step - loss: 0.2642 - accuracy: 0.8923 - val_loss: 0.2463 - val_accuracy: 0.9010 - lr: 0.0010
Epoch 3/50
2590/2590 [==============================] - 6s 2ms/step - loss: 0.2538 - accuracy: 0.8966 - val_loss: 0.2401 - val_accuracy: 0.9034 - lr: 0.0010
Epoch 4/50
2590/2590 [==============================] - 11s 4ms/step - loss: 0.2485 - accuracy: 0.8997 - val_loss: 0.2341 - val_accuracy: 0.9070 - lr: 0.0010
Epoch 5/50
2590/2590 [==============================] - 16s 6ms/step - loss: 0.2420 - accuracy: 0.9018 - val_loss: 0.2295 - val_accuracy: 0.9065 - lr: 0.0010
Epoch 6/50
2590/2590 [==============================] - 8s 3ms/step - loss: 0.2379 - accuracy: 0.9044 - val_loss: 0.2262 - val_accuracy: 0.9084 - lr: 0.0010
Epoch 7/50
2590/2590 [==============================] - 

In [138]:
#Resultados Modelo
loss = history.history['loss']
accuracy = history.history['accuracy']

print(f'Tiempo total de entrenamiento: {time_callback.total_train_time:.2f} segundos')
print(f'Pérdida: {loss[-1]}')
print(f'Accuracy Modelo: {accuracy[-1]*100}')

# Resultados Validación

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 0)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(tf.round(y_pred), 1)), dtype=tf.float32))
    return true_negatives / (true_negatives + false_positives)

# Validación con x_test

model = tf.keras.models.load_model('../MODELS/m11.h5')
y_pred = (model.predict(X_test) > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
specificity = specificity(y_test, y_pred).numpy()

print("Recall:", recall)
print("Specificity:", specificity)
print("F1-Score:", f1)
print('Precisión:', precision)

Tiempo total de entrenamiento: 333.18 segundos
Pérdida: 0.19417467713356018
Accuracy Modelo: 92.11342334747314
Recall: 0.878926528816542
Specificity: 0.5810451
F1-Score: 0.8993832440462792
Precisión: 0.9208148967551623
